In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, updatepivots!, addglobalpivots1sitesweep!, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test

In [4]:
pivotsearch = :full
seed = 123
strictlynested = false

Random.seed!(seed)

R = 20
abstol = 1e-4
δ = 10.0 / 2^R # Peaks are wider than 1/2^R.
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

rindex = [rand(1:2, R) for _ in 1:100]

f(bitlist) = any(bitlist .> 2) ? 0.0 : fx(QD.quantics_to_origcoord(grid, bitlist)[1])
rpoint = Float64[QD.quantics_to_origcoord(grid, r)[1] for r in rindex]

function fx(x)
    #res = exp(-10 * x) + 1.0/(x+1.0)
    res = exp(-10 * x)
    for r in rpoint
        res += abs(x - r) < δ ? 2 * abstol : 0.0
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
    nsearchglobalpivot=0
)

iteration = 1, rank = 1, error= 1.1102230246251565e-16, maxsamplevalue= 1.0, nglobalpivot=0
iteration = 2, rank = 1, error= 1.1102230246251565e-16, maxsamplevalue= 1.0, nglobalpivot=0
iteration = 3, rank = 1, error= 1.1102230246251565e-16, maxsamplevalue= 1.0, nglobalpivot=0


(TensorCI2{Float64} with rank 1, [1, 1, 1], [1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16])

In [5]:
TCI.addglobalpivots2sitesweep!(tci, f, rindex; fillsitetensors=false, verbosity=4)

    Computing Pi (2 x 202) at bond 1: 0.008157917 sec, LU: 2.6083e-5 sec
    Computing Pi (4 x 202) at bond 2: 0.010070834 sec, LU: 1.7583e-5 sec
    Computing Pi (8 x 202) at bond 3: 0.024280625 sec, LU: 3.3959000000000005e-5 sec
    Computing Pi (16 x 202) at bond 4: 0.042002709 sec, LU: 7.6458e-5 sec
    Computing Pi (32 x 202) at bond 5: 0.076999833 sec, LU: 0.00022300000000000003 sec
    Computing Pi (60 x 202) at bond 6: 0.146345459 sec, LU: 0.000620333 sec
    Computing Pi (100 x 202) at bond 7: 0.24564337500000002 sec, LU: 0.001463916 sec
    Computing Pi (134 x 196) at bond 8: 0.31744841700000004 sec, LU: 0.0022136250000000003 sec
    Computing Pi (164 x 194) at bond 9: 0.384104208 sec, LU: 0.0027101250000000003 sec
    Computing Pi (182 x 176) at bond 10: 0.390656875 sec, LU: 0.002784917 sec
    Computing Pi (188 x 162) at bond 11: 0.370063833 sec, LU: 0.002666958 sec
    Computing Pi (196 x 138) at bond 12: 0.32654225000000003 sec, LU: 0.002283292 sec
    Computing Pi (198 x

In [6]:
for b in 1:length(tci)-1
    @show length(tci.Iset[b+1]), length(tci.Jset[b])
end

(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (2, 485)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (4, 485)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (8, 484)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (16, 483)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (31, 477)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (51, 473)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (69, 463)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (85, 441)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (97, 398)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (101, 351)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (109, 277)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (112, 188)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (122, 113)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (146, 63)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (191, 32)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (258, 16)
(length(tci.Iset[b + 1]), length(tci.Jset[b])) = (332, 

In [7]:
TCI.fillsitetensors!(tci, f)

In [8]:
length(tci.globalpivots)

715

In [9]:
tt = TCI.TensorTrain(tci)
@show TCI.linkdims(tt)
TCI.compress!(tt, :SVD; tolerance=1e-14)
@show TCI.linkdims(tt)

TCI.linkdims(tt) = [2, 4, 8, 16, 31, 51, 69, 85, 97, 101, 109, 112, 122, 146, 191, 258, 332, 395, 446]
TCI.linkdims(tt) = [2, 4, 8, 16, 31, 51, 68, 82, 91, 93, 90, 85, 78, 60, 32, 16, 8, 4, 2]


19-element Vector{Int64}:
  2
  4
  8
 16
 31
 51
 68
 82
 91
 93
 90
 85
 78
 60
 32
 16
  8
  4
  2

In [10]:
@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex]) .> abstol) == 0

rindex_test = [rand(1:2, R) for _ in 1:100]

@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex_test]) .> abstol) == 0

Test Passed